In [1]:
import datetime as dt
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

pyo.init_notebook_mode(connected=True)
pd.options.plotting.backend = 'plotly'
dataset = pd.read_csv(r"./Datasets/DJI/DJI_MFADCCA.csv")
df = dataset.reset_index(drop=True)
df

,Unnamed: 0,Date,Open,High,Low,Close,QPLp1,QPLp2,QPLp3,QPLp4,...,QPLn19,QPLn20,QPLn21,vol,log_returns,volatility_increments,log_volumes,hurst,hurstp,hurstm
0,0,2011/1/18,2744.80,2766.17,2744.26,2765.85,2750.876351,2765.154075,2782.884506,2802.791090,...,2326.477325,2301.489354,2276.631460,500.18,0.003822,0.207066,0.058858,0.354273,0.063637,0.508620
1,1,2011/1/19,2762.81,2764.35,2717.79,2725.36,2768.926221,2783.297628,2801.144398,2821.181599,...,2341.742501,2316.590572,2291.569573,491.30,-0.014747,0.273401,-0.017913,0.177194,0.543709,-0.184073
2,2,2011/1/20,2712.40,2714.10,2686.59,2704.29,2718.404625,2732.513812,2750.034951,2769.706555,...,2299.015264,2274.322254,2249.757786,566.67,-0.007761,0.277950,0.142722,0.184256,0.598381,-0.469008
3,3,2011/1/21,2716.92,2722.74,2688.53,2689.54,2722.934631,2737.067330,2754.617667,2774.322052,...,2302.846391,2278.112232,2253.506830,521.36,-0.005469,0.080925,-0.083336,0.087697,1.264664,-1.000141
4,4,2011/1/24,2693.03,2719.73,2687.97,2717.55,2698.991744,2713.000174,2730.396190,2749.927313,...,2282.597359,2258.080689,2233.691642,431.67,0.010361,0.094394,-0.188779,0.225284,0.828188,-0.065013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1744,1744,2017/12/20,6991.26,6991.26,6935.42,6960.96,7006.737029,7043.103715,7088.264767,7138.968682,...,5925.753374,5862.106697,5798.791336,442.66,-0.000415,-0.005171,-0.151171,0.428968,0.641390,0.374092
1745,1745,2017/12/21,6972.88,6992.63,6961.20,6965.36,6988.316340,7024.587418,7069.629742,7120.200356,...,5910.174588,5846.695237,5783.546332,436.63,0.000632,-0.015317,-0.013716,0.333892,0.314275,0.306599
1746,1746,2017/12/22,6958.02,6962.26,6944.44,6959.96,6973.423444,7009.617223,7054.563557,7105.026400,...,5897.579334,5834.235265,5771.220938,372.94,-0.000776,-0.105539,-0.157669,0.286586,0.146246,0.340331
1747,1747,2017/12/26,6928.92,6942.14,6915.56,6936.25,6944.259023,6980.301432,7025.059790,7075.311586,...,5872.914335,5809.835185,5747.084398,322.15,-0.003412,-0.586730,-0.146400,0.386706,0.327121,0.161985


In [2]:
maxLevel = 21                           
maxTP = 1                
maxTS = 4096             
nTP = 0                    
p3 = 1.0 / 3.0          

stime = 0
etime = 0
Gstime = 0
Getime = 0
tlapse = 0
Gtlapse = 0

day = 2261

DT_DT= np.zeros((day,1))

DT_OP= np.zeros((day,1))
DT_HI= np.zeros((day,1))
DT_LO= np.zeros((day,1))
DT_CL= np.zeros((day,1))
DT_VL= np.zeros((day,1))

cHurst= np.zeros((day,1))
oHurst= np.zeros((day,1))
hHurst= np.zeros((day,1))
lHurst= np.zeros((day,1))
DT_RT= np.zeros((day,1))

Q= np.zeros((100,1))
NQ= np.zeros((100,1))
r= np.zeros((100,1))


QFEL = np.zeros((21,1))       
QPR = np.zeros((21,1))       
NQPR = np.zeros((21,1))      
K = np.zeros((21,1))


ALL_Pos_QPL = np.zeros((day,21))
ALL_Neg_QPL = np.zeros((day,21))

print("Printout ALL K values K0 .. K20 for first 20 Levels")
for L in range(0,21):    
    K[L] = math.pow((1.1924 + (33.2383 * L) + (56.2169 * L * L)) / (1 + (43.6106 * L)), p3)
    print("Level ",L, " K",L, " = ", K[L])   

Printout ALL K values K0 .. K20 for first 20 Levels
Level  0  K 0  =  [1.06041043]
Level  1  K 1  =  [1.2665998]
Level  2  K 2  =  [1.4911995]
Level  3  K 3  =  [1.66349934]
Level  4  K 4  =  [1.80609901]
Level  5  K 5  =  [1.92919156]
Level  6  K 6  =  [2.03832293]
Level  7  K 7  =  [2.13688129]
Level  8  K 8  =  [2.22710524]
Level  9  K 9  =  [2.31055991]
Level  10  K 10  =  [2.38838747]
Level  11  K 11  =  [2.46144992]
Level  12  K 12  =  [2.53041565]
Level  13  K 13  =  [2.59581465]
Level  14  K 14  =  [2.65807505]
Level  15  K 15  =  [2.71754825]
Level  16  K 16  =  [2.77452666]
Level  17  K 17  =  [2.82925649]
Level  18  K 18  =  [2.88194724]
Level  19  K 19  =  [2.93277882]
Level  20  K 20  =  [2.98190702]


In [3]:
DT_DT = df['Date']     
DT_OP = df['Open']
DT_HI = df['High']
DT_LO = df['Low']
DT_CL = df['Close']
hurst = df['hurst']
hurstp = df['hurstp']
hurstm = df['hurstm']
TSsize = df.shape[0]

TSsize

1749

In [4]:
# For Hurst
maxRno = TSsize - 2

for d in range (0,TSsize - 2):
    if (DT_CL[d + 1] > 0):
        DT_RT[d] = DT_CL[d] / DT_CL[d + 1]          
    else:
        DT_RT[d] = 1
        
mu = 0
for d in range(0,maxRno):
    mu = mu + DT_RT[d]
mu = mu / maxRno

sigma = 0
for d in range(0,maxRno):
    sigma = sigma + (DT_RT[d] - mu) * (DT_RT[d] - mu)
    
sigma = math.sqrt((sigma/ maxRno))

dr = 3 * sigma/ 50

auxR = 0
tQno = 0
num = 0

for nR in range (0,maxRno):    
    bFound = False
    nQ = 0
    # Get the start position of the wave function.
    auxR = 1 - (dr * 50)
    # Get the total number of the returns in each range of each segment of the wave function.
    while((bFound == False) and (nQ < 100)):
        
        if ((DT_RT[nR] > auxR) and (DT_RT[nR] <= (auxR + dr))):
            
            Q[nQ] = Q[nQ] + 1
            tQno= tQno + 1
            bFound = True
        else:
            num = num+1
            nQ = nQ + 1
            auxR = auxR + dr

auxR = 1 - (dr * 50)
for nQ in range(0,100):
    r[nQ] = auxR
    NQ[nQ] = Q[nQ] / tQno        
    auxR = auxR + dr

maxQ = 0
maxQno = 0

for nQ in range(0,100):     
    if (NQ[nQ] > maxQ):
        maxQ = NQ[nQ]
        maxQno = nQ

for i in range(0,day):     
    r0 = r[maxQno] - (dr / 2)
    r1 = r0 + dr
    rn1 = r0 - dr
    
for i in range(0,day):  
    Lup = (pow(rn1, 2) * NQ[maxQno - 1]) - (pow(r1, 2) * NQ[maxQno + 1])
    Ldw = (pow(rn1, 4) * NQ[maxQno - 1]) - (pow(r1, 4) * NQ[maxQno + 1])

L = abs(Lup / Ldw)

for eL in range(0,21):
    p = -1 * pow((2 * eL + 1), 2)
    q = -1 * L * pow((2 * eL + 1), 3) * pow(K[eL], 3)
    u = pow((-0.5 * q + np.sqrt(((q * q / 4.0) + (p * p * p / 27.0)))), p3)
    v = pow((-0.5 * q - np.sqrt(((q * q / 4.0) + (p * p * p / 27.0)))), p3)
    QFEL[eL] = u + v

for eL in range(0,21):
# Compute the QPR.  
    QPR[eL] = QFEL[eL] / QFEL[0]
#Compute the NQPR.
    NQPR[eL] = 1 + 0.21 * sigma * QPR[eL]

cQHLdata = pd.DataFrame(columns=["Date", "Open", "High", "Low", "Close", "cQHLp1", "cQHLp2", "cQHLp3", "cQHLp4", "cQHLp5", "cQHLp6", "cQHLp7", "cQHLp8",
      "cQHLp9", "cQHLp10", "cQHLp11", "cQHLp12", "cQHLp13", "cQHLp14", "cQHLp15", "cQHLp16", "cQHLp17", "cQHLp18", "cQHLp19", "cQHLp20", "cQHLp21", "cQHLn1", "cQHLn2",
      "cQHLn3", "cQHLn4", "cQHLn5", "cQHLn6", "cQHLn7", "cQHLn8", "cQHLn9", "cQHLn10", "cQHLn11", "cQHLn12", "cQHLn13", "cQHLn14", "cQHLn15", "cQHLn16",
      "cQHLn17", "cQHLn18", "cQHLn19", "cQHLn20", "cQHLn21"])

for d in range(0,TSsize):
    for eL in range(0,21):
        ALL_Pos_QPL[d][eL] = hurst[d] * NQPR[eL]
        ALL_Neg_QPL[d][eL] = hurst[d] / NQPR[eL]

cQHLdata.Date= DT_DT
cQHLdata.Open= DT_OP
cQHLdata.High= DT_HI
cQHLdata.Low = DT_LO
cQHLdata.Close = DT_CL
cqhlv = cQHLdata.values

for d in range(0,TSsize - 2):
    for eL in range(0,21):
        cqhlv[d][eL+5] = ALL_Pos_QPL[d][eL]
        cqhlv[d][eL+26] = ALL_Neg_QPL[d][eL]

cQHLdata = pd.DataFrame(cqhlv,columns=["Date", "Open", "High", "Low", "Close", "cQHLp1", "cQHLp2", "cQHLp3", "cQHLp4", "cQHLp5", "cQHLp6", "cQHLp7", "cQHLp8",
      "cQHLp9", "cQHLp10", "cQHLp11", "cQHLp12", "cQHLp13", "cQHLp14", "cQHLp15", "cQHLp16", "cQHLp17", "cQHLp18", "cQHLp19", "cQHLp20", "cQHLp21", "cQHLn1", "cQHLn2",
      "cQHLn3", "cQHLn4", "cQHLn5", "cQHLn6", "cQHLn7", "cQHLn8", "cQHLn9", "cQHLn10", "cQHLn11", "cQHLn12", "cQHLn13", "cQHLn14", "cQHLn15", "cQHLn16",
      "cQHLn17", "cQHLn18", "cQHLn19", "cQHLn20", "cQHLn21"])

dfc = pd.DataFrame(cQHLdata)
dfc.to_csv("./Datasets/DJI/DJI_qhl.csv",index = True)

In [5]:
# For hurstplus
maxRno = TSsize - 2

for d in range (0,TSsize - 2):
    if (DT_OP[d + 1] > 0):
        DT_RT[d] = DT_OP[d] / DT_OP[d + 1]          
    else:
        DT_RT[d] = 1
        
mu = 0
for d in range(0,maxRno):
    mu = mu + DT_RT[d]
mu = mu / maxRno

sigma = 0
for d in range(0,maxRno):
    sigma = sigma + (DT_RT[d] - mu) * (DT_RT[d] - mu)
    
sigma = math.sqrt((sigma/ maxRno))

dr = 3 * sigma/ 50

auxR = 0
tQno = 0
num = 0

for nR in range (0,maxRno):    
    bFound = False
    nQ = 0
    # Get the start position of the wave function.
    auxR = 1 - (dr * 50)
    # Get the total number of the returns in each range of each segment of the wave function.
    while((bFound == False) and (nQ < 100)):
        
        if ((DT_RT[nR] > auxR) and (DT_RT[nR] <= (auxR + dr))):
            
            Q[nQ] = Q[nQ] + 1
            tQno= tQno + 1
            bFound = True
        else:
            num = num+1
            nQ = nQ + 1
            auxR = auxR + dr

auxR = 1 - (dr * 50)
for nQ in range(0,100):
    r[nQ] = auxR
    NQ[nQ] = Q[nQ] / tQno        
    auxR = auxR + dr

maxQ = 0
maxQno = 0

for nQ in range(0,100):     
    if (NQ[nQ] > maxQ):
        maxQ = NQ[nQ]
        maxQno = nQ

for i in range(0,day):     
    r0 = r[maxQno] - (dr / 2)
    r1 = r0 + dr
    rn1 = r0 - dr
    
for i in range(0,day):  
    Lup = (pow(rn1, 2) * NQ[maxQno - 1]) - (pow(r1, 2) * NQ[maxQno + 1])
    Ldw = (pow(rn1, 4) * NQ[maxQno - 1]) - (pow(r1, 4) * NQ[maxQno + 1])

L = abs(Lup / Ldw)

for eL in range(0,21):
    p = -1 * pow((2 * eL + 1), 2)
    q = -1 * L * pow((2 * eL + 1), 3) * pow(K[eL], 3)
    u = pow((-0.5 * q + np.sqrt(((q * q / 4.0) + (p * p * p / 27.0)))), p3)
    v = pow((-0.5 * q - np.sqrt(((q * q / 4.0) + (p * p * p / 27.0)))), p3)
    QFEL[eL] = u + v

for eL in range(0,21):
# Compute the QPR.  
    QPR[eL] = QFEL[eL] / QFEL[0]
#Compute the NQPR.
    NQPR[eL] = 1 + 0.21 * sigma * QPR[eL]

cQHLdata = pd.DataFrame(columns=["Date", "Open", "High", "Low", "Close", "oQHLp1", "oQHLp2", "oQHLp3", "oQHLp4", "oQHLp5", "oQHLp6", "oQHLp7", "oQHLp8",
      "oQHLp9", "oQHLp10", "oQHLp11", "oQHLp12", "oQHLp13", "oQHLp14", "oQHLp15", "oQHLp16", "oQHLp17", "oQHLp18", "oQHLp19", "oQHLp20", "oQHLp21", "oQHLn1", "oQHLn2",
      "oQHLn3", "oQHLn4", "oQHLn5", "oQHLn6", "oQHLn7", "oQHLn8", "oQHLn9", "oQHLn10", "oQHLn11", "oQHLn12", "oQHLn13", "oQHLn14", "oQHLn15", "oQHLn16",
      "oQHLn17", "oQHLn18", "oQHLn19", "oQHLn20", "oQHLn21"])

for d in range(0,TSsize):
    for eL in range(0,21):
        ALL_Pos_QPL[d][eL] = hurstp[d] * NQPR[eL]
        ALL_Neg_QPL[d][eL] = hurstp[d] / NQPR[eL]

cQHLdata.Date= DT_DT
cQHLdata.Open= DT_OP
cQHLdata.High= DT_HI
cQHLdata.Low = DT_LO
cQHLdata.Close = DT_CL
cqhlv = cQHLdata.values

for d in range(0,TSsize - 2):
    for eL in range(0,21):
        cqhlv[d][eL+5] = ALL_Pos_QPL[d][eL]
        cqhlv[d][eL+26] = ALL_Neg_QPL[d][eL]

cQHLdata = pd.DataFrame(cqhlv,columns=["Date", "Open", "High", "Low", "Close", "oQHLp1", "oQHLp2", "oQHLp3", "oQHLp4", "oQHLp5", "oQHLp6", "oQHLp7", "oQHLp8",
      "oQHLp9", "oQHLp10", "oQHLp11", "oQHLp12", "oQHLp13", "oQHLp14", "oQHLp15", "oQHLp16", "oQHLp17", "oQHLp18", "oQHLp19", "oQHLp20", "oQHLp21", "oQHLn1", "oQHLn2",
      "oQHLn3", "oQHLn4", "oQHLn5", "oQHLn6", "oQHLn7", "oQHLn8", "oQHLn9", "oQHLn10", "oQHLn11", "oQHLn12", "oQHLn13", "oQHLn14", "oQHLn15", "oQHLn16",
      "oQHLn17", "oQHLn18", "oQHLn19", "oQHLn20", "oQHLn21"])

dfc = pd.DataFrame(cQHLdata)
dfc.to_csv("./Datasets/DJI/DJI_qhlp.csv",index = True)

In [6]:
# For hurstminus
maxRno = TSsize - 2

for d in range (0,TSsize - 2):
    if (DT_HI[d + 1] > 0):
        DT_RT[d] = DT_HI[d] / DT_HI[d + 1]          
    else:
        DT_RT[d] = 1
        
mu = 0
for d in range(0,maxRno):
    mu = mu + DT_RT[d]
mu = mu / maxRno

sigma = 0
for d in range(0,maxRno):
    sigma = sigma + (DT_RT[d] - mu) * (DT_RT[d] - mu)
    
sigma = math.sqrt((sigma/ maxRno))

dr = 3 * sigma/ 50

auxR = 0
tQno = 0
num = 0

for nR in range (0,maxRno):    
    bFound = False
    nQ = 0
    # Get the start position of the wave function.
    auxR = 1 - (dr * 50)
    # Get the total number of the returns in each range of each segment of the wave function.
    while((bFound == False) and (nQ < 100)):
        
        if ((DT_RT[nR] > auxR) and (DT_RT[nR] <= (auxR + dr))):
            
            Q[nQ] = Q[nQ] + 1
            tQno= tQno + 1
            bFound = True
        else:
            num = num+1
            nQ = nQ + 1
            auxR = auxR + dr

auxR = 1 - (dr * 50)
for nQ in range(0,100):
    r[nQ] = auxR
    NQ[nQ] = Q[nQ] / tQno        
    auxR = auxR + dr

maxQ = 0
maxQno = 0

for nQ in range(0,100):     
    if (NQ[nQ] > maxQ):
        maxQ = NQ[nQ]
        maxQno = nQ

for i in range(0,day):     
    r0 = r[maxQno] - (dr / 2)
    r1 = r0 + dr
    rn1 = r0 - dr
    
for i in range(0,day):  
    Lup = (pow(rn1, 2) * NQ[maxQno - 1]) - (pow(r1, 2) * NQ[maxQno + 1])
    Ldw = (pow(rn1, 4) * NQ[maxQno - 1]) - (pow(r1, 4) * NQ[maxQno + 1])

L = abs(Lup / Ldw)

for eL in range(0,21):
    p = -1 * pow((2 * eL + 1), 2)
    q = -1 * L * pow((2 * eL + 1), 3) * pow(K[eL], 3)
    u = pow((-0.5 * q + np.sqrt(((q * q / 4.0) + (p * p * p / 27.0)))), p3)
    v = pow((-0.5 * q - np.sqrt(((q * q / 4.0) + (p * p * p / 27.0)))), p3)
    QFEL[eL] = u + v

for eL in range(0,21):
# Compute the QPR.  
    QPR[eL] = QFEL[eL] / QFEL[0]
#Compute the NQPR.
    NQPR[eL] = 1 + 0.21 * sigma * QPR[eL]

cQHLdata = pd.DataFrame(columns=["Date", "Open", "High", "Low", "Close", "hQHLp1", "hQHLp2", "hQHLp3", "hQHLp4", "hQHLp5", "hQHLp6", "hQHLp7", "hQHLp8",
      "hQHLp9", "hQHLp10", "hQHLp11", "hQHLp12", "hQHLp13", "hQHLp14", "hQHLp15", "hQHLp16", "hQHLp17", "hQHLp18", "hQHLp19", "hQHLp20", "hQHLp21", "hQHLn1", "hQHLn2",
      "hQHLn3", "hQHLn4", "hQHLn5", "hQHLn6", "hQHLn7", "hQHLn8", "hQHLn9", "hQHLn10", "hQHLn11", "hQHLn12", "hQHLn13", "hQHLn14", "hQHLn15", "hQHLn16",
      "hQHLn17", "hQHLn18", "hQHLn19", "hQHLn20", "hQHLn21"])

for d in range(0,TSsize):
    for eL in range(0,21):
        ALL_Pos_QPL[d][eL] = hurstm[d] * NQPR[eL]
        ALL_Neg_QPL[d][eL] = hurstm[d] / NQPR[eL]

cQHLdata.Date= DT_DT
cQHLdata.Open= DT_OP
cQHLdata.High= DT_HI
cQHLdata.Low = DT_LO
cQHLdata.Close = DT_CL
cqhlv = cQHLdata.values

for d in range(0,TSsize - 2):
    for eL in range(0,21):
        cqhlv[d][eL+5] = ALL_Pos_QPL[d][eL]
        cqhlv[d][eL+26] = ALL_Neg_QPL[d][eL]

cQHLdata = pd.DataFrame(cqhlv,columns=["Date", "Open", "High", "Low", "Close", "hQHLp1", "hQHLp2", "hQHLp3", "hQHLp4", "hQHLp5", "hQHLp6", "hQHLp7", "hQHLp8",
      "hQHLp9", "hQHLp10", "hQHLp11", "hQHLp12", "hQHLp13", "hQHLp14", "hQHLp15", "hQHLp16", "hQHLp17", "hQHLp18", "hQHLp19", "hQHLp20", "hQHLp21", "hQHLn1", "hQHLn2",
      "hQHLn3", "hQHLn4", "hQHLn5", "hQHLn6", "hQHLn7", "hQHLn8", "hQHLn9", "hQHLn10", "hQHLn11", "hQHLn12", "hQHLn13", "hQHLn14", "hQHLn15", "hQHLn16",
      "hQHLn17", "hQHLn18", "hQHLn19", "hQHLn20", "hQHLn21"])

dfc = pd.DataFrame(cQHLdata)
dfc.to_csv("./Datasets/DJI/DJI_qhlm.csv",index = True)